
# 🎨 Visualisation des données — Seaborn (statistiques & styles haut-niveau)
_Master IA-GI — Notebook 4 (Partie 2)_

Ce notebook complète Matplotlib en s'appuyant sur **Seaborn** pour des graphiques statistiques haut-niveau (thèmes, palettes, facettes, estimateurs).  
On y retrouve des **exemples concrets**, des **exercices avec hints/solutions**, et un **mini-projet** d'EDA rapide.

**Objectifs d’apprentissage**
- Utiliser `seaborn` avec les **datasets intégrés** (tips, penguins, iris, flights)
- Maîtriser les **graphiques relationnels** (`scatterplot`, `lineplot`) et **catégoriels** (`barplot`, `countplot`, `boxplot`, `violinplot`, `swarmplot`)
- Visualiser les **distributions** (`histplot`, `kdeplot`, `ecdfplot`, `jointplot`, `pairplot`)
- Réaliser des **régressions** (`regplot`, `lmplot`)
- Construire des **facettes** (`FacetGrid`, `relplot`, `catplot`)
- Exploiter le **styling** (thèmes, contextes, palettes) et intégrer avec Matplotlib pour exporter des figures propres

**Pré-requis** : Matplotlib (Partie 1)  
**Durée estimée** : 3h



---
## 0) ⚙️ Préparation


In [ ]:

import numpy as np
import pandas as pd

# Installer seaborn si besoin (en Colab)
try:
    import seaborn as sns  # noqa: F401
except Exception:
    !pip -q install seaborn

import seaborn as sns
import matplotlib.pyplot as plt

print("Seaborn:", sns.__version__, "| Pandas:", pd.__version__)
sns.set_theme()  # thème par défaut agréable



---
## 1) Datasets intégrés & aperçu rapide
> Seaborn propose des datasets pédagogiques : `tips`, `penguins`, `iris`, `flights`, `titanic`, `diamonds`, etc.


In [ ]:

tips = sns.load_dataset("tips")
penguins = sns.load_dataset("penguins")
iris = sns.load_dataset("iris")
flights = sns.load_dataset("flights")

tips.head(), penguins.head(), iris.head(), flights.head()



---
## 2) Graphiques relationnels (`scatterplot`, `lineplot`)


In [ ]:

# Scatter: relation total_bill vs tip, styled by sex & time
ax = sns.scatterplot(data=tips, x="total_bill", y="tip", hue="sex", style="time")
ax.set_title("Pourboire en fonction de l'addition")
plt.tight_layout()


In [ ]:

# Line: moyenne mensuelle des passagers (flights), avec intervalle de confiance
df = flights.pivot(index="month", columns="year", values="passengers").reset_index().melt(id_vars="month", var_name="year", value_name="passengers")
ax = sns.lineplot(data=df, x="month", y="passengers", hue="year", estimator=None)
ax.set_title("Passagers par mois et par année")
plt.xticks(rotation=45)
plt.tight_layout()



**Exercice 2.1 — Scatter coloré**
Avec `penguins`, trace `bill_length_mm` vs `bill_depth_mm` coloré par `species` et pointillé par `sex` (si dispo).

<details>
<summary>✅ Solution</summary>

```python
ax = sns.scatterplot(data=penguins, x="bill_length_mm", y="bill_depth_mm", hue="species", style="sex")
ax.set_title("Morphologie des manchots")
plt.tight_layout()
```
</details>



---
## 3) Graphiques catégoriels (`barplot`, `countplot`, `boxplot`, `violinplot`, `swarmplot`)


In [ ]:

# barplot (avec estimateur par défaut = moyenne, IC 95%)
ax = sns.barplot(data=tips, x="day", y="tip", hue="sex")
ax.set_title("Tip moyen par jour et par sexe")
plt.tight_layout()


In [ ]:

# countplot (effectifs par catégorie)
ax = sns.countplot(data=tips, x="day", hue="time")
ax.set_title("Nombre d'observations par jour et service")
plt.tight_layout()


In [ ]:

fig, axes = plt.subplots(1, 3, figsize=(12, 4), sharey=True)
sns.boxplot(data=tips, x="day", y="total_bill", ax=axes[0])
axes[0].set_title("Boxplot")
sns.violinplot(data=tips, x="day", y="total_bill", ax=axes[1])
axes[1].set_title("Violinplot")
sns.swarmplot(data=tips, x="day", y="total_bill", ax=axes[2], size=3)
axes[2].set_title("Swarmplot")
fig.suptitle("Distributions par jour")
fig.tight_layout()



**Exercice 3.1 — Ordre & palette**  
Refais un `barplot` de `total_bill` par `day` **trié** par moyenne décroissante et applique une **palette** colorée.

<details>
<summary>💡 Hint</summary>
Utilise `order=...` après avoir calculé des moyennes, et `palette="viridis"` (par ex.).
</details>
<details>
<summary>✅ Solution</summary>

```python
order = tips.groupby("day")["total_bill"].mean().sort_values(ascending=False).index
ax = sns.barplot(data=tips, x="day", y="total_bill", order=order, palette="viridis")
ax.set_title("Addition moyenne par jour (triée)")
plt.tight_layout()
```
</details>



---
## 4) Distributions (`histplot`, `kdeplot`, `ecdfplot`, `jointplot`, `pairplot`)


In [ ]:

fig, axes = plt.subplots(1, 3, figsize=(12, 3.5))
sns.histplot(tips, x="total_bill", kde=True, ax=axes[0])
axes[0].set_title("Hist + KDE")

sns.ecdfplot(data=tips, x="total_bill", ax=axes[1])
axes[1].set_title("ECDF")

# jointplot retourne sa propre figure
g = sns.jointplot(data=tips, x="total_bill", y="tip", kind="hex")
g.fig.suptitle("Jointplot hex", y=1.02)

plt.tight_layout()


In [ ]:

# Pairplot : matrice de scatter/diag distributions par espèce
g = sns.pairplot(penguins.dropna(), hue="species", corner=True, diag_kind="kde")
g.fig.suptitle("Pairplot Penguins", y=1.02)



---
## 5) Régression (`regplot`, `lmplot`)


In [ ]:

ax = sns.regplot(data=tips, x="total_bill", y="tip")
ax.set_title("Régression linéaire: tip ~ total_bill")
plt.tight_layout()


In [ ]:

# lmplot = FacetGrid + regplot (supporte hue/col/row)
g = sns.lmplot(data=tips, x="total_bill", y="tip", hue="sex", col="time")
g.fig.suptitle("lmplot avec facettes", y=1.02)



---
## 6) Matrices : `heatmap`, `clustermap`


In [ ]:

# Corrélation sur iris
corr = iris.drop(columns=["species"]).corr(numeric_only=True)
ax = sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
ax.set_title("Corrélation (iris)")
plt.tight_layout()


In [ ]:

# Clustermap sur les passagers par mois/année
fl = flights.pivot(index="month", columns="year", values="passengers")
g = sns.clustermap(fl, cmap="mako", standard_scale=0)
g.fig.suptitle("Clustermap Flights", y=1.02)



---
## 7) Facettes : `FacetGrid`, `relplot`, `catplot`


In [ ]:

g = sns.FacetGrid(tips, col="time", row="smoker", margin_titles=True)
g.map_dataframe(sns.scatterplot, x="total_bill", y="tip", alpha=0.7)
g.add_legend()
g.fig.suptitle("FacetGrid: tips", y=1.02)


In [ ]:

# Interfaces haut-niveau relplot/catplot
g = sns.relplot(data=penguins, x="bill_length_mm", y="bill_depth_mm", hue="species", col="island")
g.fig.suptitle("relplot Penguins", y=1.02)

g2 = sns.catplot(data=tips, x="day", y="total_bill", hue="sex", kind="violin")
g2.fig.suptitle("catplot Violin", y=1.02)



---
## 8) Thèmes, palettes, contextes (`set_theme`, `set_style`, `set_context`)


In [ ]:

# Styles disponibles : white, dark, whitegrid, darkgrid, ticks
sns.set_style("whitegrid")
sns.set_context("talk")  # tailles adaptées aux slides
sns.set_palette("deep")

ax = sns.barplot(data=tips, x="day", y="tip", hue="sex")
ax.set_title("Style whitegrid + context talk")
plt.tight_layout()

# Reset (optionnel) : sns.reset_defaults(); sns.set_theme()



**Exercice 8.1 — Thème personnalisé**  
Applique `set_style("ticks")`, `set_context("paper")`, et une palette `"rocket"` puis refais un `scatterplot` des `penguins` coloré par `species`.

<details>
<summary>✅ Solution</summary>

```python
sns.set_style("ticks")
sns.set_context("paper")
sns.set_palette("rocket")
ax = sns.scatterplot(data=penguins, x="bill_length_mm", y="bill_depth_mm", hue="species")
ax.set_title("Penguins (style perso)")
plt.tight_layout()
```
</details>



---
## 9) Intégration Matplotlib & export propre
- Seaborn retourne des **Axes/figures Matplotlib** ➜ on peut `set_title`, `set_xlabel`, `fig.savefig(...)`


In [ ]:

ax = sns.histplot(tips, x="total_bill", kde=True)
ax.set(title="Distribution des additions", xlabel="Total bill ($)", ylabel="Fréquence")
ax.figure.tight_layout()
ax.figure.savefig("seaborn_hist.png", dpi=300, bbox_inches="tight")
"Fichier enregistré: seaborn_hist.png"



---
## 10) Bonnes pratiques (récap)
- **Toujours** vérifier les **données manquantes/outliers** avant d'interpréter des graphes
- Utiliser `hue`, `style`, `size` avec parcimonie (lisibilité > gadgets)
- Facetter (`col`/`row`) pour comparer des sous-populations
- Harmoniser le **style** (thème + palette) et exporter en haute qualité



---
## 11) 📝 Quiz express
1) Quelle différence entre `pairplot` et `jointplot` ?  
2) Comment afficher un **IC** sur un `barplot` ?  
3) Quelle API permet de facetter automatiquement des `scatterplot` ?  
4) Comment changer rapidement le **thème** global ?  
5) À quoi sert `clustermap` ?

<details>
<summary>✅ Corrigé</summary>

1) `pairplot` = matrice de relations entre **plusieurs** variables ; `jointplot` = relation **bivariée** avec marges.  
2) C'est implicite (moyenne ± IC) — on peut ajuster avec `errorbar`/`ci` selon version.  
3) `relplot` (ou `FacetGrid`).  
4) `sns.set_theme()` (ou `set_style`/`set_context`).  
5) Clustering + heatmap pour visualiser structures/similarités.
</details>



---
## 12) 🎯 Mini-projet — *EDA express “tips” & “penguins”*
**Tâche** : produire une **fiche EDA** composée de 4 à 6 figures Seaborn bien mises en page :
1) **Distribution** clé (hist+KDE) avec titres/labels propres  
2) **Graphique catégoriel** pertinent (moyenne/IC)  
3) **Relation bivariée** (scatter avec hue + éventuellement régression)  
4) **Facettes** sur 2 dimensions (ex. `time` × `smoker` ou `species` × `island`)  
5) **Heatmap** de corrélations (si num.)  
6) Export en PNG 300 dpi

> Exige style cohérent (`set_context("talk")`, `set_style("whitegrid")`, palette choisie).  
> L’idée est de rendre un “mini-rapport” visuel clair et autoportant.


In [ ]:

# Exemple de squelette (à adapter)
sns.set_theme()
sns.set_style("whitegrid")
sns.set_context("talk")
sns.set_palette("deep")

# 1) Distribution
fig1, ax1 = plt.subplots(figsize=(6,4))
sns.histplot(tips, x="total_bill", kde=True, ax=ax1)
ax1.set(title="Distribution des additions", xlabel="Total bill ($)")
fig1.tight_layout()
fig1.savefig("eda1_distribution.png", dpi=300, bbox_inches="tight")

# 2) Catégoriel
fig2, ax2 = plt.subplots(figsize=(6,4))
sns.barplot(data=tips, x="day", y="tip", hue="sex", ax=ax2)
ax2.set(title="Tips par jour et par sexe", xlabel="Jour", ylabel="Tip")
fig2.tight_layout()
fig2.savefig("eda2_categoriel.png", dpi=300, bbox_inches="tight")

# 3) Relation bivariée + régression
fig3 = sns.lmplot(data=tips, x="total_bill", y="tip", hue="sex", col="time")
fig3.fig.suptitle("Tips vs Total bill (lmplot)", y=1.02)
fig3.figure.savefig("eda3_regression.png", dpi=300, bbox_inches="tight")

# 4) Facettes Penguins
g = sns.relplot(data=penguins.dropna(), x="bill_length_mm", y="bill_depth_mm", hue="species", col="island")
g.fig.suptitle("Penguins par île", y=1.02)
g.figure.savefig("eda4_facets_penguins.png", dpi=300, bbox_inches="tight")

# 5) Heatmap corr Iris
corr = iris.drop(columns=["species"]).corr(numeric_only=True)
fig5, ax5 = plt.subplots(figsize=(5,4))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="magma", square=True, ax=ax5)
ax5.set_title("Corrélations Iris")
fig5.tight_layout()
fig5.savefig("eda5_heatmap_iris.png", dpi=300, bbox_inches="tight")

"Exports: eda1_distribution.png, eda2_categoriel.png, eda3_regression.png, eda4_facets_penguins.png, eda5_heatmap_iris.png"



---
## 📚 Ressources
- Seaborn docs : https://seaborn.pydata.org/  
- Tutorials : https://seaborn.pydata.org/tutorial.html  
- Figure-level vs Axes-level : https://seaborn.pydata.org/tutorial/function_overview.html  

**Suite** : **SciPy** puis **Pandas** (dans la progression du module).
